In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 350.6 kB/s eta 0:00:0000:0100:16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 423.3 kB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317129 sha256=701080811e7cc553837150f0875a755eb35cdc14532b5bcd71d16300cc24f1f8
  Stored in directory: /Users/hitegarg/Library/Caches/pip/wheels/db/1f/dc/108f626bfc37b270c82f400eb9cc6028ad9501cacad8dcce15
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkConf, SparkContext
import collections

In [3]:
# conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
# sc = SparkContext(conf = conf)
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/17 12:59:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!{"head -n 5 ml-100k/u.data"}

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596


In [5]:
lines = sc.textFile("ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("(%s, %i)" % (key, value))


(1, 6110)
(2, 11370)
(3, 27145)
(4, 34174)
(5, 21201)


----------

In [6]:
def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    return (age, numFriends)

In [7]:
lines = sc.textFile("fakefriends.csv")
rdd = lines.map(parseLine)
totalsByAge = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByAge = totalsByAge.mapValues(lambda x: x[0] / x[1])
results = averagesByAge.collect()
for result in results:
    print(result)

(26, 242.05882352941177)
(40, 250.8235294117647)
(68, 269.6)
(54, 278.0769230769231)
(38, 193.53333333333333)
(56, 306.6666666666667)
(36, 246.6)
(22, 206.42857142857142)
(60, 202.71428571428572)
(30, 235.8181818181818)
(42, 303.5)
(48, 281.4)
(50, 254.6)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(44, 282.1666666666667)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(18, 343.375)
(34, 245.5)
(33, 325.3333333333333)
(55, 295.53846153846155)
(59, 220.0)
(37, 249.33333333333334)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(43, 230.57142857142858)
(35, 211.625)
(45, 309.53846153846155)
(67, 214.625)
(19, 213.27272727272728)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(49, 184.66666666666666)
(39, 169.28571428571428)
(31, 267.25)
(41, 268.55555555555554)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(29, 215.91666666666666)
(47, 233

-------

In [8]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

In [12]:
lines = sc.textFile("1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F


---------

In [25]:
input = sc.textFile("book.txt")
words = input.flatMap(lambda x: x.split(" ")).map(lambda x: x.upper())
wordCounts = words.countByValue()

for word, count in wordCounts.items():
    cleanWord = word.encode('ascii', 'ignore')
    if (cleanWord):
        print(cleanWord.decode() + " " + str(count))

SELF-EMPLOYMENT: 1
BUILDING 32
AN 178
INTERNET 13
BUSINESS 290
OF 960
ONE 81
ACHIEVING 1
FINANCIAL 17
AND 923
PERSONAL 47
FREEDOM 33
THROUGH 57
A 1187
LIFESTYLE 38
TECHNOLOGY 9
BY 118
FRANK 10
KANE 7
COPYRIGHT 3
2015 3
KANE. 1
ALL 122
RIGHTS 3
RESERVED 2
WORLDWIDE. 2
CONTENTS 1
DISCLAIMER 2
PREFACE 2
PART 32
I: 2
MAKING 23
THE 1282
BIG 38
DECISION 9
OVERCOMING 2
INERTIA 2
FEAR 2
FAILURE 3
CAREER 20
INDOCTRINATION 4
CARROT 4
ON 401
STICK 5
EGO 3
PROTECTION 4
YOUR 1416
EMPLOYER 28
AS 343
SECURITY 7
BLANKET 2
WHY 22
ITS 35
WORTH 34
IT 357
UNLIMITED 5
GROWTH 31
POTENTIAL 32
INVESTING 16
IN 594
YOURSELF, 14
NOT 178
SOMEONE 60
ELSE 16
NO 76
DEPENDENCIES 5
COMMUTE 9
TO 1801
LIVE 22
WHERE 52
YOU 1415
WANT 107
WORK 120
WHEN 102
HOW 157
IS 549
SELF-EMPLOYMENT 44
FOR 522
YOU? 6
FLOWCHART: 2
SHOULD 67
I 322
EVEN 104
CONSIDER 26
SELF-EMPLOYMENT? 3
HAVING 30
SAFETY 7
NET 13
PLANNING 14
HEALTH 31
CARE 22
SELF-ASSESSMENT 4
QUIZ 2
II: 2
HAPPEN 6
DESIGNING 4
FALLACY 2
INTRODUCING 3
IDEAL 3
CASE 12
STUDY

In [26]:
import re

In [30]:
def normaliseWords(word):
    return re.compile(r'\W+',re.UNICODE).split(word.lower())

In [31]:
input = sc.textFile("book.txt")
words = input.flatMap(normaliseWords)
wordCounts = words.countByValue()

for word, count in wordCounts.items():
    cleanWord = word.encode('ascii', 'ignore')
    if (cleanWord):
        print(cleanWord.decode() + " " + str(count))

self 111
employment 75
building 33
an 178
internet 26
business 383
of 970
one 100
achieving 1
financial 17
and 934
personal 48
freedom 41
through 57
a 1191
lifestyle 44
technology 11
by 122
frank 11
kane 10
copyright 3
2015 4
all 137
rights 3
reserved 2
worldwide 4
contents 1
disclaimer 2
preface 2
part 33
i 387
making 25
the 1292
big 42
decision 12
overcoming 2
inertia 2
fear 3
failure 3
career 31
indoctrination 5
carrot 4
on 428
stick 6
ego 3
protection 7
your 1420
employer 44
as 343
security 8
blanket 2
why 25
it 649
s 391
worth 39
unlimited 6
growth 39
potential 38
investing 16
in 616
yourself 78
not 203
someone 62
else 33
no 76
dependencies 6
commute 14
to 1828
live 25
where 53
you 1878
want 122
work 144
when 102
how 163
is 560
for 537
flowchart 4
should 69
even 104
consider 26
having 30
safety 7
net 13
planning 16
health 35
care 24
assessment 4
quiz 4
ii 2
happen 13
designing 4
fallacy 2
introducing 3
ideal 3
case 26
study 4
sundog 24
software 60
other 78
ideas 27
key 6
points 5


In [53]:
input = sc.textFile("book.txt")
words = input.flatMap(normaliseWords)

wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey()
results = wordCountsSorted.collect()

for result in results[::-1]:
    count = str(result[0])
    word = result[1].encode('ascii', 'ignore')
    if (word):
        print(word.decode() + ":\t" + count)

you:	1878
to:	1828
your:	1420
the:	1292
a:	1191
of:	970
and:	934
that:	747
it:	649
in:	616
is:	560
for:	537
on:	428
are:	424
if:	411
s:	391
i:	387
business:	383
can:	376
be:	369
as:	343
have:	321
with:	315
t:	301
this:	280
or:	278
time:	255
but:	242
they:	234
will:	231
what:	229
at:	220
my:	215
re:	214
do:	207
not:	203
about:	202
more:	200
product:	182
an:	178
up:	177
need:	174
from:	166
them:	166
how:	163
there:	162
out:	161
new:	153
people:	145
work:	144
so:	143
just:	142
own:	140
all:	137
don:	133
customers:	123
get:	123
their:	122
company:	122
want:	122
by:	122
some:	121
ll:	114
self:	111
website:	109
make:	108
may:	107
even:	104
when:	102
one:	100
ve:	95
than:	92
also:	91
much:	90
job:	90
who:	88
money:	86
was:	85
these:	82
find:	81
sales:	80
into:	79
only:	79
yourself:	78
like:	78
other:	78
probably:	76
no:	76
employment:	75
ads:	75
day:	73
good:	72
many:	71
ad:	70
might:	70
most:	70
before:	70
should:	69
those:	68
products:	67
well:	66
market:	66
still:	65
sure:	65
plan:	64
goog